In [ ]:
import os
import re
import json
import time
import urllib.parse
import pandas as pd
from tqdm import tqdm
from selenium import webdriver
from selenium.common.exceptions import TimeoutException

In [ ]:
# READ JSON
# https://stackoverflow.com/questions/20199126/reading-json-from-a-file
DIR_PATH = 'data'
FILENAME = 'regulation_data_modified.json'
file_path = os.path.join(DIR_PATH, FILENAME)
with open(file_path) as file:
    json_data = json.load(file)

display(len(json_data))
display(json_data)

In [ ]:
def download_all_pdf(data: list[dict], full_download_dir_path: str):
    
    relative_download_dir_path = os.path.relpath(full_download_dir_path, start=os.getcwd())
    os.makedirs(relative_download_dir_path, exist_ok=True)

    # https://stackoverflow.com/questions/60170311/how-to-switch-download-directory-using-selenium-firefox-python
    firefox_options = webdriver.FirefoxOptions()
    firefox_options.set_preference("browser.download.folderList", 2)  # Gunakan folder custom (2)
    firefox_options.set_preference("browser.download.dir", full_download_dir_path)  # Tentukan folder tujuan
    firefox_options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf")  # Hindari prompt
    firefox_options.set_preference("pdfjs.disabled", True)  # Nonaktifkan preview PDF di browser
    # firefox_options.add_argument('-headless')

    driver = webdriver.Firefox(firefox_options)
    driver.set_page_load_timeout(5)

    for row in tqdm(iterable=data, desc='Download files'):
        try:
            driver.get(row['download_link'])
        except TimeoutException as _:
            pass

    time.sleep(10)

    for row in tqdm(iterable=data, desc='Rename files'):
        # https://stackoverflow.com/questions/300445/how-to-unquote-a-urlencoded-unicode-string-in-python
        downloaded_file = re.search(r'\d+\/(.*\.pdf)$', row['download_link'])[1]
        downloaded_file = urllib.parse.unquote(downloaded_file)
        downloaded_file = os.path.join(relative_download_dir_path, downloaded_file)
        renamed_file = os.path.join(relative_download_dir_path, row['download_name'] + '.pdf')

        if os.path.exists(downloaded_file):
            os.rename(downloaded_file, renamed_file)
        else:
            print(f'File {downloaded_file} not found.')
    
    time.sleep(10)
    
    driver.quit()

In [ ]:
# Folder tujuan untuk download
full_download_dir_path = os.path.join(os.getcwd(), 'data', 'pdf')
download_all_pdf(json_data, full_download_dir_path)